In [1]:
import pathlib

import pandas as pd
from pycytominer import feature_select
from pycytominer.cyto_utils import output

In [2]:
# set path to normalized data
normalized_data_path = pathlib.Path(
    "../data/CP_scDINO_features/combined_norm_data.parquet"
).resolve(strict=True)

# set the outout file path
feature_selected_output_file_path = pathlib.Path(
    "../data/CP_scDINO_features/combined_norm_fs_data.parquet"
).resolve()

# read in the normalized data
normalized_data = pd.read_parquet(normalized_data_path)
print(normalized_data.shape)
normalized_data.head()

(184115, 3869)


,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,...,channel_DNA_cls_feature_91_scDINO,channel_DNA_cls_feature_92_scDINO,channel_DNA_cls_feature_93_scDINO,channel_DNA_cls_feature_94_scDINO,channel_DNA_cls_feature_95_scDINO,channel_DNA_cls_feature_96_scDINO,channel_DNA_cls_feature_97_scDINO,channel_DNA_cls_feature_98_scDINO,channel_DNA_cls_feature_99_scDINO,channel_DNA_cls_feature_9_scDINO
0,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,11,...,-0.734818,0.654381,-0.070655,0.813982,0.296032,-0.227765,1.785438,0.158278,-0.762728,0.725202
1,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,15,...,-0.468204,-0.666395,-0.128430,1.562595,0.045222,-0.489645,0.273701,-0.211752,-0.189933,0.283274
2,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,14,...,1.703526,-0.181057,0.025402,0.039498,-0.683365,-1.985459,-0.149541,-0.449522,0.561193,0.198248
3,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,26,...,1.238245,-1.402979,0.801880,0.694113,-0.436960,-0.348364,-2.542127,0.954522,0.149704,-0.263583
4,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,35,...,-1.899088,0.255286,0.305438,0.016286,0.422010,-0.642979,2.355769,-1.516771,0.829755,2.041394


In [3]:
# define operations to be performed on the data
# list of operations for feature select function to use on input profile
feature_select_ops = [
    "variance_threshold",
    "blocklist",
    "drop_na_columns",
    "correlation_threshold",
]

# Get columns that contain "Metadata"
metadata_features = normalized_data.columns[
    normalized_data.columns.str.contains("Metadata")
].tolist()

# get the feature columns
feature_columns = normalized_data.columns.difference(metadata_features).to_list()

In [4]:
manual_block_list = [
    "Nuclei_AreaShape_BoundingBoxArea",
    "Nuclei_AreaShape_BoundingBoxMinimum_X",
    "Cells_AreaShape_BoundingBoxArea",
]

In [5]:
feature_select_df = feature_select(
    normalized_data,
    operation=feature_select_ops,
    features=feature_columns,
)
# add "Metadata_" to the beginning of each column name in the list
feature_select_df.columns = [
    "Metadata_" + column if column in manual_block_list else column
    for column in feature_select_df.columns
]
print("Feature selection complete, saving to parquet file!")
# save features selected df as parquet file
output(
    df=feature_select_df,
    output_filename=feature_selected_output_file_path,
    output_type="parquet",
)
print("Features have been selected!")
# check to see if the shape of the df has changed indicating feature selection occurred
print(normalized_data.shape)
print(feature_select_df.shape)
print(f"{normalized_data.shape[1] - feature_select_df.shape[1]} features were removed.")
print(f"{feature_select_df.shape[1]} features remain.")
feature_select_df.head()

Feature selection complete, saving to parquet file!
Features have been selected!
(184115, 3869)
(184115, 2375)
1494 features were removed.
2375 features remain.


,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,...,channel_DNA_cls_feature_91_scDINO,channel_DNA_cls_feature_92_scDINO,channel_DNA_cls_feature_93_scDINO,channel_DNA_cls_feature_94_scDINO,channel_DNA_cls_feature_95_scDINO,channel_DNA_cls_feature_96_scDINO,channel_DNA_cls_feature_97_scDINO,channel_DNA_cls_feature_98_scDINO,channel_DNA_cls_feature_99_scDINO,channel_DNA_cls_feature_9_scDINO
0,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,11,...,-0.734818,0.654381,-0.070655,0.813982,0.296032,-0.227765,1.785438,0.158278,-0.762728,0.725202
1,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,15,...,-0.468204,-0.666395,-0.128430,1.562595,0.045222,-0.489645,0.273701,-0.211752,-0.189933,0.283274
2,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,14,...,1.703526,-0.181057,0.025402,0.039498,-0.683365,-1.985459,-0.149541,-0.449522,0.561193,0.198248
3,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,26,...,1.238245,-1.402979,0.801880,0.694113,-0.436960,-0.348364,-2.542127,0.954522,0.149704,-0.263583
4,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,35,...,-1.899088,0.255286,0.305438,0.016286,0.422010,-0.642979,2.355769,-1.516771,0.829755,2.041394
